# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [5]:
import numpy as np
from scipy import optimize
import sympy as sm
import ipywidgets as widgets # Interactive plots
import matplotlib.pyplot as plt

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject as mod

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Layout
2. Analytical solution - Use standard solution - Sunday
3. Numerical - Find output - Sunday
4. Two extension - 1. CES utility functions (Tuesday). 2. More consumers (Tuesday). (3. Combination?) (4. More goods)
5. Conclusion (Thursday)

# 1. Model description of baseline model

**Time:** One period.

**Demographics:** Initially one consumer and one firm. The firm is owned by the consumer and therefore the income also depends on the profit. We do not assume initial goods endowments. 

**Household:** For our baseline model we assume that the consumer has Cobb-Douglas preferences, where the two goods are the product (c) and leisure (l)
$$
\begin{aligned}
& max u =c^{\alpha}*l^{1-\alpha}, 0<\alpha<1\\
 & \text{s.t.}\\
 & I=\Pi + w*L=p*c +w*l \\

\end{aligned}
$$

p is the price of the good, w is the wage, L is the initial labor endowment, l is leisure and $\Pi$ is the profit. We have assumed monotonicity. Further, we have the firm problem: 
$$
\begin{aligned}
&\Pi =p*y-w*h

\end{aligned}
$$

The production function will be defined as follow, where the interpretation of A can be TFP: 

$$
\begin{aligned}
&y =A*h^{\beta}, \beta <1 \\
&max \Pi =p*A*h^{\beta}-w*h

\end{aligned}
$$

We assume the initial labor endowment is 24 hours, i.e. a day. We can therefore define the following relationsship in equilibrium: 
$$
\begin{aligned}
&L=24=l+h

\end{aligned}
$$




**Equilibrium:** The procedure is defined where the labor market (l+h=24) and good market (y=c) clear. Further, the firm has maximized its profit and the consumer has maximized its utility. 



## 2. Analytical solution

The baseline model can be solved analytically. The starting point is to derive the firm's demand for labor. This is done by taking the FOC of the profit w.r.t. working hours. This given the following result:  

$$
\begin{aligned}
& h^*=(\frac{\beta*p*A}{w})^{\frac{1}{1-\beta}}
\end{aligned}
$$

This gives the following output and profit: 

$$
\begin{aligned}
& y^*=A*(\frac{\beta*p*A}{w})^{\frac{\beta}{1-\beta}}\\
& \Pi^*=p*A*(\frac{\beta*p*A}{w})^{\frac{\beta}{1-\beta}}-w(\frac{\beta*p*A}{w})^{\frac{1}{1-\beta}}
\end{aligned}
$$

Now, we will solve for the consumer's problem. It should be noted that the working hours and profit is exogenous for the consumer as the firm has already decided this based on the prices. We have a Cobb-Douglas function and can therefore use the standard solution: 

$$
\begin{aligned}
& c^*=\alpha*\frac {I}{p}\\
& l^*=(1-\alpha)*\frac{I}{w}
\end{aligned}
$$

Now we want to find the prices in equilibrium. Lets sets the wage as numeraire, i.e. w=1. We can then solve for the labor market: 

$$
\begin{aligned}
& l^*=24-h^*\\
& \leftrightarrow (1-\alpha)*(p*A*(\beta*p*A)^{\frac{\beta}{1-\beta}}-(\beta*p*A)^{\frac{1}{1-\beta}}+24)=24-(\beta*p*A)^{\frac{1}{1-\beta}}
\end{aligned}
$$

Solving this for p gives: 

$$
\begin{aligned}
& p^*=\frac {24^{1-\beta}}{\beta*A}+(\frac {24\alpha}{1-\alpha})^{1-\beta}*\frac {1}{A*\beta^\beta}
\end{aligned}
$$

We see that higher marginal output elasticity ($\beta$) or higher productivity (A) decreases the price, which makes intuively sense as this will increase the supply. Further, higher demand ($\alpha$) will increase the price, which again makes intuitively sense. With this price we have the general solution, which can be plotted below. 

In [6]:
mod.consumption(0.5,0.5,5)

AssertionError: Good market does not clear

Now we have the general solution. This can be illustrated by using specific values for the parameters in the model. 

In [3]:
# Write out which arguments to interactive_figure you want to be changing or staying fixed 
widgets.interact(mod.interactive_figure,
    alpha2=widgets.FloatSlider(description=r"alpha2", min=0.1, max=0.9, step=0.05, value=0.5), #Cannot be 1, because both alphas are 1 then. cannot be 0 either
    e11=widgets.FloatSlider(description=r"e11", min=0.0, max=10, step=1, value=5),
    e21=widgets.FloatSlider(description=r"e21", min=0.0, max=10, step=1, value=5),
    e12=widgets.FloatSlider(description=r"e12", min=0.0, max=10, step=1, value=5),
    e22=widgets.FloatSlider(description=r"e22", min=0.0, max=10, step=1, value=5),
);

interactive(children=(FloatSlider(value=0.5, description='alpha2', max=0.9, min=0.1, step=0.05), FloatSlider(v…

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 